In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import torch
import torch.nn as nn

In [7]:
mnist = loadmat('mnist_49_3000.mat')
X = mnist['x']
y = mnist['y'][0]
X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=1000, random_state=42)
print(X_train.shape)
print(y_train.shape)

(2000, 784)
(2000,)


In [8]:
def converter(inp):
    if inp > 0:
        return 1
    else:
        return 0

y_train_mod = [converter(val) for val in y_train]
y_test_mod  = [converter(val) for val in y_test]

In [9]:
class LogisticRegression(nn.Module):
    def __init__(self, x_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(x_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x)) 

In [11]:
# Gradient descent parameters
learning_rate = 0.01
lamb = 0.01
num_iter = 1000

# Initialize the model, loss function, and optimizer
model = LogisticRegression(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=lamb, lr=learning_rate)
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train_mod).float()
for epoch in range(num_iter):
    y_pred = model(X_train)
    print(y_pred.shape)
    print(y_train.shape)
    loss = criterion(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

torch.Size([2000, 1])
torch.Size([2000])


C:\Users\Austin Gardiner\AppData\Local\Temp\ipykernel_8008\3209492409.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()


RuntimeError: expected scalar type Long but found Float